<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/Sessio%232%20-%205000-10000%20-%20Multiprocessing%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install statsmodels --upgrade
!pip install -U lingam
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import math
import plotly.graph_objects as go
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed, Input
from keras.models import Model
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
import ast
from statsmodels.tsa.ar_model import AutoReg
from sklearn.metrics import mean_squared_error
from math import sqrt
from matplotlib import pyplot
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LassoCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import TimeSeriesSplit
from numpy import arange
import warnings
warnings.filterwarnings('ignore')
#from keras.utils import plot_model
#import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import grangercausalitytests



df=pd.read_csv(r'/content/drive/MyDrive/PHD/2021/household_power_consumption.txt', sep=';', header=0, low_memory=False, infer_datetime_format=True, parse_dates={'datetime':[0,1]}, index_col=['datetime'])

df.head()

#df = df[['Global_active_power','Global_reactive_power','Global_intensity']]
#Imputing NULL
df = df.replace('?', np.nan)
df.isnull().sum()

#IMputing missing value

def fill_missing(values):
    one_day = 60*24
    for row in range(df.shape[0]):
        for col in range(df.shape[1]):
            if np.isnan(values[row][col]):
                values[row,col] = values[row-one_day,col]
df = df.astype('float32')
fill_missing(df.values)
df.isnull().sum()


#Downsampling to Hours now  --------------------------- ----------------------------------------------------------------------------------------------------------------------------------####
daily_df = df.resample('H').sum()
daily_df.head()
#daily_df =df
ts_len = daily_df.shape[0]

#Now convert index to column
daily_df['datetime']=daily_df.index


#remove index column unless required.

daily_df.drop(daily_df.columns[7], axis=1, inplace=True)
#daily_df.drop(daily_df.columns[2], axis=1, inplace=True) #dropped Voltage
#Scaling the values
whole_series = daily_df
scalers={}
for i in daily_df.columns:
    scaler = MinMaxScaler(feature_range=(-1,1))
    s_s = scaler.fit_transform(whole_series[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    whole_series[i]=s_s



#### GRANGER CAUSALITY PROVES that Voltage is strongly related with most - so we are removing voltage############

whole_series.drop(whole_series.columns[2], axis=1, inplace=True)
######## SPLIT THE TIME SERIES INTO LONG SEQUENCES OF LENGTH K

#Creating subsequences


def extract_windows_vectorized(array, large_seq_size):
    start = 0
    last_index = len(array)-1
    max_time =  last_index - large_seq_size +1  ##last index upto which sliding windoe begining can go

    sub_windows = (
        start +
        # expand_dims are used to convert a 1D array to 2D array.
        np.expand_dims(np.arange(large_seq_size), 0) +
        np.expand_dims(np.arange(max_time + 1), 0).T
    ).astype(int)

    return array[sub_windows]


#DEFINE K - User parameter - length of the LONG time series sequence.
K = 120  #taking 5 days of hourly data

large_seq_size = K
n_features = whole_series.shape[1]

Long = extract_windows_vectorized(whole_series.values,large_seq_size) #Shape: (1263, 180, 7) - 902 long sequences, each 180 long, 7 variables

maxval = Long.shape[0]
count_train = int(math.ceil(0.9*maxval))
Long_train = Long[0:count_train]
Long_test = Long[count_train:]

#First, define a function that can generate small subsequences for all the large K sequences

def generate_small_seq(series, n_past, n_future):
  #
  # n_past ==> no of past observations -- OR -- sliding window
  #
  # n_future ==> no of future observations -- prediction variable y
  #
  X, y = list(), list()
  for window_start in range(len(series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(series):
      break
    # slicing the past and future parts of the window
    past, future = series[window_start:past_end, :], series[past_end:future_end, :]
    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)


n_future=1
predictions = list()

rmse_list = list()
min_window_list = list()
best_window_for_long_seq = list()
AIC = list()
n_fold = 6 # 5 fold plus 1 for test)
# evaluate a logistic regression model using k-fold cross-validation

#-------------------Multiprocessing----------------------------------------------------------

#!/usr/bin/env python3
import itertools
import multiprocessing

#A function which will process a tuple of parameters
def iosw_gen(params):
  i = params[0]
  j = params[1]
  k = params[2]
  best_window_for_long_seq.clear()
  Total_small_seq_X, Total_small_seq_y = generate_small_seq(Long_train[i,:,:],k, n_future)
  Total_small_seq_X_reshaped = Total_small_seq_X[:,:,j].reshape(Total_small_seq_X.shape[0],Total_small_seq_X.shape[1])
  Total_small_seq_y_reshaped = Total_small_seq_y[:,:,j].reshape(Total_small_seq_y.shape[0],Total_small_seq_y.shape[1])
  maxval2 = Total_small_seq_X_reshaped.shape[0]
  count_train2 = int(math.ceil(0.7*maxval2))
  Small_train_X = Total_small_seq_X_reshaped[0:count_train2]
  Small_train_y = Total_small_seq_y_reshaped[0:count_train2]
  Small_test_X =  Total_small_seq_X_reshaped[count_train2:]
  Small_test_y = Total_small_seq_y_reshaped[count_train2:]
  tscv = TimeSeriesSplit(n_splits=n_fold-1)
  regressor = LassoCV(alphas=arange(0, 1, 0.01), cv=tscv, n_jobs=-1)
  try:
    regressor.fit(Small_train_X, Small_train_y) #get jth time series only
    y_predj = regressor.predict(Small_test_X)
       #print('alpha: %f' % regressor.alpha_)
    rmse = sqrt(mean_squared_error(Small_test_y, y_predj))
  except:
    rmse = 9999
  print("done this")
  return i,j,k,rmse


!pip install ipython-autotime
%load_ext autotime


#Generate values for each parameter
i = range(20001,20501)
j = range(Long_train.shape[2])
#j = range(5)
k = range(2,(round(K)-n_fold-1))

#Generate a list of tuples where each tuple is a combination of parameters.
#The list will contain all possible combinations of parameters.
paramlist = list(itertools.product(i,j,k))

#Generate processes equal to the number of cores
pool = multiprocessing.Pool()

#cores = multiprocessing.cpu_count() # Count the number of cores in a computer
#cores

#Distribute the parameter sets evenly across the cores

res  = pool.map(iosw_gen,paramlist)
res_df = pd.DataFrame(res, columns=["i","j","k","rmse"])
res_df_iosw = res_df.loc[res_df.groupby(["i","j"]).rmse.idxmin()]
print("done")
np.save(r'/content/drive/MyDrive/PHD/2024/iosw_df-20000-20500-S.npy',res_df_iosw)
print("saved upto 20500")

#Generate values for each parameter
i = range(20501,21001)
j = range(Long_train.shape[2])
#j = range(5)
k = range(2,(round(K)-n_fold-1))

#Generate a list of tuples where each tuple is a combination of parameters.
#The list will contain all possible combinations of parameters.
paramlist = list(itertools.product(i,j,k))

#Generate processes equal to the number of cores
pool = multiprocessing.Pool()

#cores = multiprocessing.cpu_count() # Count the number of cores in a computer
#cores

#Distribute the parameter sets evenly across the cores

res  = pool.map(iosw_gen,paramlist)
res_df = pd.DataFrame(res, columns=["i","j","k","rmse"])
res_df_iosw = res_df.loc[res_df.groupby(["i","j"]).rmse.idxmin()]
print("done")
np.save(r'/content/drive/MyDrive/PHD/2024/iosw_df-20500-21000-S.npy',res_df_iosw)
print("saved upto 21000")

#Generate values for each parameter
i = range(21001,21501)
j = range(Long_train.shape[2])
#j = range(5)
k = range(2,(round(K)-n_fold-1))

#Generate a list of tuples where each tuple is a combination of parameters.
#The list will contain all possible combinations of parameters.
paramlist = list(itertools.product(i,j,k))

#Generate processes equal to the number of cores
pool = multiprocessing.Pool()

#cores = multiprocessing.cpu_count() # Count the number of cores in a computer
#cores

#Distribute the parameter sets evenly across the cores

res  = pool.map(iosw_gen,paramlist)
res_df = pd.DataFrame(res, columns=["i","j","k","rmse"])
res_df_iosw = res_df.loc[res_df.groupby(["i","j"]).rmse.idxmin()]
print("done")
np.save(r'/content/drive/MyDrive/PHD/2024/iosw_df-21000-21500-S.npy',res_df_iosw)
print("saved upto 21500")


#Generate values for each parameter
i = range(21501,22001)
j = range(Long_train.shape[2])
#j = range(5)
k = range(2,(round(K)-n_fold-1))

#Generate a list of tuples where each tuple is a combination of parameters.
#The list will contain all possible combinations of parameters.
paramlist = list(itertools.product(i,j,k))

#Generate processes equal to the number of cores
pool = multiprocessing.Pool()

#cores = multiprocessing.cpu_count() # Count the number of cores in a computer
#cores

#Distribute the parameter sets evenly across the cores

res  = pool.map(iosw_gen,paramlist)
res_df = pd.DataFrame(res, columns=["i","j","k","rmse"])
res_df_iosw = res_df.loc[res_df.groupby(["i","j"]).rmse.idxmin()]
print("done")
np.save(r'/content/drive/MyDrive/PHD/2024/iosw_df-21500-22000-S.npy',res_df_iosw)
print("saved upto 22000")

Streaming output truncated to the last 5000 lines.
done this
done this
done thisdone this

done thisdone this

done this
done thisdone thisdone this


done thisdone this
done this
done this

done thisdone this

done this
done this
done this
done this
done thisdone this

done this
done thisdone this
done this

done thisdone this

done this
done thisdone this

done this
done this
done this
done this
done this
done this
done this
done this
done this
done this
done thisdone this

done thisdone this

done this
done thisdone this

done this
done thisdone this

done thisdone thisdone this


done this
done this
done thisdone this

done thisdone this

done this
done thisdone this
done this

done thisdone thisdone this


done this
done this
done this
done this
done thisdone this

done this
done this
done this
done thisdone this

done this
done this
done this
done thisdone thisdone this


done this
done thisdone this
done thisdone this


done this
done thisdone this
done this

done this
done this

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# New Section